# 2024-03-01 - IdentifyingContext Explorations


## Imports

In [2]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
from typing import Any, Dict, Union, List
from collections import defaultdict

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
import IPython
from IPython.core.formatters import PlainTextFormatter
from IPython import get_ipython

from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()
# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from neuropy.utils.misc import numpyify_array
from neuropy.utils.result_context import IdentifyingContext
from neuropy.utils.result_context import IdentifyingContext as Ctx
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.utils.mixins.diffable import OrderedSet

annotations_man = UserAnnotationsManager()
user_annotations = annotations_man.get_user_annotations()
original_user_annotations = deepcopy(user_annotations)

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
## Find the longest common string for all annotations

In [ ]:
print(IdentifyingContext.find_unique_values(user_annotations))
# {'format_name': {'kdiba'}, 'animal': {'vvp01', 'pin01', 'gor01'}, 'exper_name': {'one', 'two'}, 'session_name': {'11-02_19-28-0', '2006-4-09_16-40-54', '2006-4-09_17-29-30', '2006-6-09_1-22-43', '2006-4-10_12-58-3', '2006-6-12_15-55-31', 'fet11-01_12-58-54', '11-03_12-3-25', '11-02_17-46-44', '2006-6-12_16-53-46', '2006-6-08_14-26-15', '2006-4-10_12-25-50', '2006-6-09_22-24-40', '2006-6-08_21-16-25', '2006-6-07_16-40-19'}, 'display_fn_name': {'DecodedEpochSlices'}, 'epochs': {'ripple', 'replays'}, 'decoder': {'short_LR', 'long_RL', 'long_LR', 'short_RL', 'long_results_obj', 'short_results_obj'}, 'user_annotation': {'selections'}}


In [ ]:
a_dict = {IdentifyingContext(k1='a', k2='b'): 'good'}
assert a_dict[IdentifyingContext(k2='b', k1='a')] == 'good'

In [ ]:
assert IdentifyingContext(k1='a', k2='b') == IdentifyingContext(k2='b', k1='a')

In [3]:
user_annotations = {}
# user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')] = [4, 22, 24, 28, 30, 38, 42, 50, 55, 60, 67, 70, 76, 83, 85, 100, 103, 107, 108, 113, 118, 121, 122, 131, 140, 142, 149, 153, 170, 171]
# user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj',user_annotation='selections')] = [2, 7, 11, 17, 20, 22, 30, 34, 38, 39, 41, 43, 47, 49, 55, 59, 60, 69, 70, 75, 77, 80, 83, 85, 86, 100, 107, 110, 113, 114, 115, 118, 120, 121, 122, 126, 130, 131, 138, 140, 142, 149, 157, 160, 168, 170]        
# user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='long_LR',user_annotation='selections')] = [[380.739, 380.865], [550.845, 551.034], [600.244, 600.768], [1431.7, 1431.87], [2121.38, 2121.72]]
# user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='long_RL',user_annotation='selections')] = [[1202.96, 1203.26], [1433.42, 1433.58], [1600.77, 1601.16], [1679.18, 1679.68]]
# user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='short_LR',user_annotation='selections')] = [[551.872, 552.328], [565.161, 565.417], [616.348, 616.665], [919.581, 919.692], [1149.57, 1149.8], [1167.82, 1168.17], [1384.71, 1385.01], [1424.02, 1424.22], [1446.52, 1446.65], [1538.1, 1538.48], [1690.72, 1690.82], [1820.96, 1821.29], [1979.72, 1979.86], [1995.48, 1995.95], [2121.38, 2121.72], [2267.05, 2267.41]]
# user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='short_RL',user_annotation='selections')] = [[373.508, 373.754], [391.895, 392.163], [600.244, 600.768], [1015.26, 1015.5], [1079.9, 1080.08], [1310.59, 1310.92], [1433.42, 1433.58], [1494.95, 1495.4], [1558.22, 1558.42], [1616.92, 1617.09], [1774.48, 1774.61], [1956.96, 1957.2], [2011.36, 2011.54], [2059.35, 2059.56], [2074.35, 2074.62], [2156.53, 2156.79], [2233.53, 2233.95], [2260.49, 2260.61], [2521.1, 2521.31]]

user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')] = [1, 3, 11, 13, 14, 15, 17, 21, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 36, 37, 39, 42, 43, 44, 45, 46, 48, 51, 52, 53, 55, 57, 58, 60, 61, 62, 68, 69, 70, 72, 74, 76, 81, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 97, 98, 100, 101, 105, 106, 109, 112, 113, 114, 115, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 130, 131, 132]
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj',user_annotation='selections')] = [2, 3, 4, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 36, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49, 51, 53, 55, 63, 64, 66, 67, 69, 70, 72, 75, 77, 78, 80, 81, 83, 84, 85, 86, 87, 88, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 126, 127, 131, 132]
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='long_LR',user_annotation='selections')] = [[181.692, 181.9], [188.797, 189.046], [193.648, 193.893], [210.712, 211.049], [218.107, 218.507], [241.692, 241.846], [282.873, 283.142], [869.784, 869.936], [1285.37, 1285.51], [1306.52, 1306.78], [1338, 1338.13], [1492.93, 1493.02], [1493.48, 1493.69]]
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='long_RL',user_annotation='selections')] = [[64.8766, 65.1232], [240.488, 240.772], [398.601, 399.047], [1152.56, 1152.76], [1367.65, 1367.73], [1368.48, 1368.85]]
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='short_LR',user_annotation='selections')] = [[61.3971, 61.6621], [72.6069, 72.9543], [77.7351, 78.0483], [1378.88, 1379.02], [1485.89, 1486.15], [1492.93, 1493.02], [1493.48, 1493.69], [1530.55, 1530.79], [1807.34, 1807.48], [1832.06, 1832.19], [1832.54, 1832.61], [1848.99, 1849.22], [1865.27, 1865.45], [1866.81, 1867.07], [1998.45, 1998.57]]
user_annotations[IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',epochs='ripple',decoder='short_RL',user_annotation='selections')] = [[41.0119, 41.3591], [303.683, 303.898], [1513.62, 1513.77], [1519.64, 1519.79], [1633.03, 1633.27], [1892.27, 1892.52], [2051.14, 2051.27]]



unique_values_dict = IdentifyingContext.find_unique_values(user_annotations)
# unique_values_dict = find_unique_values(user_annotations)
unique_values_dict


{'format_name': ['kdiba'],
 'animal': ['gor01'],
 'exper_name': ['one'],
 'session_name': ['2006-6-08_14-26-15'],
 'display_fn_name': ['DecodedEpochSlices'],
 'epochs': ['replays', 'ripple'],
 'decoder': ['long_results_obj',
  'short_results_obj',
  'long_LR',
  'long_RL',
  'short_LR',
  'short_RL'],
 'user_annotation': ['selections']}

In [6]:


# def get_nested_initialization_code_strings(self, subset_includelist=None, subset_excludelist=None) -> str:
#     """ returns the string that contains valid code to initialize a matching object. """
#     init_args_list_str = ",".join([f"{k}='{v}'" for k,v in self.to_dict(subset_includelist=subset_includelist, subset_excludelist=subset_excludelist).items()]) # "format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'"
#     return f"IdentifyingContext({init_args_list_str})" #"IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')"

def build_new_nested_context_str(common_context, user_annotations):
    """ Builds a nested hierarchy of annotations like:
        with IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',user_annotation='selections') as ctx:
            with (ctx + IdentifyingContext(epochs='replays')) as ctx:
                user_annotations[ctx + Ctx(decoder='long_results_obj')] = [5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123]
                user_annotations[ctx + Ctx(decoder='short_results_obj')] = [ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131]
            with (ctx + IdentifyingContext(epochs='ripple')) as ctx:
                user_annotations[ctx + Ctx(decoder='long_LR')] = [[292.624, 292.808], [304.44, 304.656], [380.746, 380.904], [873.001, 873.269], [953.942, 954.258], [2212.47, 2212.54], [2214.24, 2214.44], [2214.65, 2214.68], [2219.73, 2219.87], [2422.6, 2422.82], [2451.06, 2451.23], [2452.07, 2452.22], [2453.38, 2453.55], [2470.82, 2470.97], [2473, 2473.15]]
                user_annotations[ctx + Ctx(decoder='long_RL')] = [[487.205, 487.451], [518.52, 518.992], [802.912, 803.114], [803.592, 803.901], [804.192, 804.338], [831.621, 831.91], [893.989, 894.103], [982.605, 982.909], [1034.82, 1034.86], [1035.12, 1035.31], [1200.7, 1200.9], [1273.35, 1273.54], [1274.12, 1274.44], [1380.75, 1380.89], [1448.17, 1448.34], [1746.25, 1746.43], [1871, 1871.22], [2050.89, 2050.99], [2051.25, 2051.68]]
                user_annotations[ctx + Ctx(decoder='short_LR')] = [[876.27, 876.452], [950.183, 950.448], [953.942, 954.258], [1044.95, 1045.45], [1129.65, 1129.84], [1259.29, 1259.44], [1259.72, 1259.88], [1511.2, 1511.43], [1511.97, 1512.06], [1549.24, 1549.37], [1558.47, 1558.68], [1560.66, 1560.75], [1561.31, 1561.41], [1561.82, 1561.89], [1655.99, 1656.21], [1730.89, 1731.07], [1734.81, 1734.95], [1861.41, 1861.53], [1909.78, 1910.04], [1967.74, 1968.09], [2036.97, 2037.33], [2038.03, 2038.27], [2038.53, 2038.73], [2042.39, 2042.64], [2070.82, 2071.03], [2153.03, 2153.14], [2191.26, 2191.39], [2192.12, 2192.36], [2193.78, 2193.99], [2194.56, 2194.76], [2200.65, 2200.8], [2201.85, 2202.03], [2219.73, 2219.87], [2248.61, 2248.81], [2249.7, 2249.92], [2313.89, 2314.06], [2422.6, 2422.82], [2462.67, 2462.74], [2482.13, 2482.61], [2484.41, 2484.48], [2530.72, 2530.92], [2531.22, 2531.3], [2556.11, 2556.38], [2556.6, 2556.92]]
                user_annotations[ctx + Ctx(decoder='short_RL')] = [[66.6616, 66.779], [888.227, 888.465], [890.87, 891.037], [910.571, 911.048], [1014.1, 1014.28], [1200.7, 1200.9], [1211.21, 1211.33], [1214.61, 1214.83], [1317.71, 1318.22], [1333.49, 1333.69], [1380.75, 1380.89], [1381.96, 1382.32], [1448.17, 1448.34], [1499.59, 1499.71], [1744.34, 1744.59], [1798.64, 1798.77], [1970.81, 1970.95], [1994.07, 1994.25], [2050.89, 2050.99], [2051.25, 2051.68], [2132.66, 2132.98], [2203.73, 2203.82], [2204.54, 2204.66], [2317.03, 2317.12], [2330.01, 2330.16], [2331.84, 2331.96], [2403.11, 2403.41], [2456.24, 2456.33], [2456.47, 2456.57], [2457.49, 2458.01]]

    """
    def _indent_str(an_indent_level: int) -> str:
        return "\t" * an_indent_level
    
    def _with_block_template(an_indent_level: int, ctxt):
        # global indent_level
        return f"{_indent_str(an_indent_level)}with {ctxt.get_initialization_code_string(class_name_override='Ctx')} as ctx:"
    def _sub_ctxt_block_template(an_indent_level: int, ctxt):
        # global indent_level
        # indent_level = indent_level + 1
        return f"{_indent_str(an_indent_level)}with (ctx + {ctxt.get_initialization_code_string(class_name_override='Ctx')}) as ctx:"
    def _leaf_ctxt_assignment_template(an_indent_level: int, ctxt, value):
        # indent_level = indent_level + 1
        return f"{_indent_str(an_indent_level)}user_annotations[ctx + {ctxt.get_initialization_code_string(class_name_override='Ctx')}] = {list(v)}"

    indent_level: int = 0
    code_strs: List[str] = []
    code_str: str = ""

    code_strs.append(_with_block_template(indent_level, common_context))
    indent_level = indent_level + 1
    common_context_user_annotations = IdentifyingContext.converting_to_relative_contexts(common_context, user_annotations)
    for k, v in common_context_user_annotations.items():
        code_strs.append(_leaf_ctxt_assignment_template(indent_level, k, v))

    code_str = code_str + '\n'.join(code_strs)
    return code_str

"""
with IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',user_annotation='selections') as ctx:
    with (ctx + IdentifyingContext(epochs='replays')) as ctx:
        user_annotations[ctx + Ctx(decoder='long_results_obj')] = [5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123]
        user_annotations[ctx + Ctx(decoder='short_results_obj')] = [ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131]
    with (ctx + IdentifyingContext(epochs='ripple')) as ctx:
        user_annotations[ctx + Ctx(decoder='long_LR')] = [[292.624, 292.808], [304.44, 304.656], [380.746, 380.904], [873.001, 873.269], [953.942, 954.258], [2212.47, 2212.54], [2214.24, 2214.44], [2214.65, 2214.68], [2219.73, 2219.87], [2422.6, 2422.82], [2451.06, 2451.23], [2452.07, 2452.22], [2453.38, 2453.55], [2470.82, 2470.97], [2473, 2473.15]]
        user_annotations[ctx + Ctx(decoder='long_RL')] = [[487.205, 487.451], [518.52, 518.992], [802.912, 803.114], [803.592, 803.901], [804.192, 804.338], [831.621, 831.91], [893.989, 894.103], [982.605, 982.909], [1034.82, 1034.86], [1035.12, 1035.31], [1200.7, 1200.9], [1273.35, 1273.54], [1274.12, 1274.44], [1380.75, 1380.89], [1448.17, 1448.34], [1746.25, 1746.43], [1871, 1871.22], [2050.89, 2050.99], [2051.25, 2051.68]]
        user_annotations[ctx + Ctx(decoder='short_LR')] = [[876.27, 876.452], [950.183, 950.448], [953.942, 954.258], [1044.95, 1045.45], [1129.65, 1129.84], [1259.29, 1259.44], [1259.72, 1259.88], [1511.2, 1511.43], [1511.97, 1512.06], [1549.24, 1549.37], [1558.47, 1558.68], [1560.66, 1560.75], [1561.31, 1561.41], [1561.82, 1561.89], [1655.99, 1656.21], [1730.89, 1731.07], [1734.81, 1734.95], [1861.41, 1861.53], [1909.78, 1910.04], [1967.74, 1968.09], [2036.97, 2037.33], [2038.03, 2038.27], [2038.53, 2038.73], [2042.39, 2042.64], [2070.82, 2071.03], [2153.03, 2153.14], [2191.26, 2191.39], [2192.12, 2192.36], [2193.78, 2193.99], [2194.56, 2194.76], [2200.65, 2200.8], [2201.85, 2202.03], [2219.73, 2219.87], [2248.61, 2248.81], [2249.7, 2249.92], [2313.89, 2314.06], [2422.6, 2422.82], [2462.67, 2462.74], [2482.13, 2482.61], [2484.41, 2484.48], [2530.72, 2530.92], [2531.22, 2531.3], [2556.11, 2556.38], [2556.6, 2556.92]]
        user_annotations[ctx + Ctx(decoder='short_RL')] = [[66.6616, 66.779], [888.227, 888.465], [890.87, 891.037], [910.571, 911.048], [1014.1, 1014.28], [1200.7, 1200.9], [1211.21, 1211.33], [1214.61, 1214.83], [1317.71, 1318.22], [1333.49, 1333.69], [1380.75, 1380.89], [1381.96, 1382.32], [1448.17, 1448.34], [1499.59, 1499.71], [1744.34, 1744.59], [1798.64, 1798.77], [1970.81, 1970.95], [1994.07, 1994.25], [2050.89, 2050.99], [2051.25, 2051.68], [2132.66, 2132.98], [2203.73, 2203.82], [2204.54, 2204.66], [2317.03, 2317.12], [2330.01, 2330.16], [2331.84, 2331.96], [2403.11, 2403.41], [2456.24, 2456.33], [2456.47, 2456.57], [2457.49, 2458.01]]

"""

common_context = IdentifyingContext.find_longest_common_context(user_annotations)
common_context_user_annotations = IdentifyingContext.converting_to_relative_contexts(common_context, user_annotations)
# common_context_user_annotations

# code_str: str = ""

# common_context.get_initialization_code_string()

code_str: str = build_new_nested_context_str(common_context, user_annotations=user_annotations)
print(code_str)




"\nwith IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19',display_fn_name='DecodedEpochSlices',user_annotation='selections') as ctx:\n    with (ctx + IdentifyingContext(epochs='replays')) as ctx:\n        user_annotations[ctx + Ctx(decoder='long_results_obj')] = [5,  13,  15,  17,  20,  21,  24,  31,  33,  43,  44,  49,  63, 64,  66,  68,  70,  71,  74,  76,  77,  78,  84,  90,  94,  95, 104, 105, 122, 123]\n        user_annotations[ctx + Ctx(decoder='short_results_obj')] = [ 12,  13,  15,  17,  20,  24,  30,  31,  32,  33,  41,  43,  49, 54,  55,  68,  70,  71,  73,  76,  77,  78,  84,  89,  94, 100, 104, 105, 111, 114, 115, 117, 118, 122, 123, 131]\n    with (ctx + IdentifyingContext(epochs='ripple')) as ctx:\n        user_annotations[ctx + Ctx(decoder='long_LR')] = [[292.624, 292.808], [304.44, 304.656], [380.746, 380.904], [873.001, 873.269], [953.942, 954.258], [2212.47, 2212.54], [2214.24, 2214.44], [2214.65, 2214.68], [2219.

with Ctx(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15',display_fn_name='DecodedEpochSlices',user_annotation='selections') as ctx:
	user_annotations[ctx + Ctx(epochs='replays',decoder='long_results_obj')] = [1, 3, 11, 13, 14, 15, 17, 21, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 36, 37, 39, 42, 43, 44, 45, 46, 48, 51, 52, 53, 55, 57, 58, 60, 61, 62, 68, 69, 70, 72, 74, 76, 81, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 97, 98, 100, 101, 105, 106, 109, 112, 113, 114, 115, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 130, 131, 132]
	user_annotations[ctx + Ctx(epochs='replays',decoder='short_results_obj')] = [2, 3, 4, 8, 9, 10, 11, 13, 14, 15, 16, 17, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 36, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49, 51, 53, 55, 63, 64, 66, 67, 69, 70, 72, 75, 77, 78, 80, 81, 83, 84, 85, 86, 87, 88, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 12

In [ ]:
non_leaf_unique_values = {k:v[0] for k, v in unique_values_dict.items() if len(v) == 1}
non_leaf_unique_values
leaf_unique_values = {k:v for k, v in unique_values_dict.items() if len(v) > 1}
leaf_unique_values


In [ ]:
## Find longest common string:
# non_leaf_unique_values

common_context = IdentifyingContext(**non_leaf_unique_values)
common_context

In [ ]:
matching_entries = IdentifyingContext.matching(user_annotations, criteria=non_leaf_unique_values)
# relevant_entries

relative_contexts_dict = {}
for a_ctx, v in matching_entries.items():
    a_relative_context = a_ctx - common_context
    relative_contexts_dict[a_relative_context] = v

relative_contexts_dict

In [ ]:

# {'format_name': {'kdiba'},
#  'animal': {'gor01'},
#  'exper_name': {'two'},
#  'session_name': {'2006-6-09_22-24-40'},
#  'display_fn_name': {'DecodedEpochSlices'},
#  'epochs': {'ripple'},
#  'decoder': {'long_LR', 'long_RL', 'short_LR', 'short_RL'},
#  'user_annotation': {'selections'}}


# {'format_name': {'kdiba'},
#  'animal': {'gor01'},
#  'exper_name': {'two'},
#  'session_name': {'2006-6-09_22-24-40'},
#  'display_fn_name': {'DecodedEpochSlices'},
#  'epochs': {'replays', 'ripple'},
#  'decoder': {'long_LR',
#   'long_RL',
#   'long_results_obj',
#   'short_LR',
#   'short_RL',
#   'short_results_obj'},
#  'user_annotation': {'selections'}}


In [ ]:

# with IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43',display_fn_name='DecodedEpochSlices',user_annotation='selections') as ctx:
#     print(f'ctx: {ctx}')
#     # inner_ctx = ctx.overwriting_context(epochs='ripple', decoder='short_RL')
#     inner_ctx = ctx + Ctx(epochs='ripple', decoder='short_RL')
#     print(f'inner_ctx: {inner_ctx}')


with IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43',display_fn_name='DecodedEpochSlices',user_annotation='selections') as ctx:
    with (ctx + IdentifyingContext(epochs='ripple')) as ctx:
        user_annotations[ctx + Ctx(decoder='long_LR')] = np.array([array([132.511, 132.791]), array([149.959, 150.254]), array([1186.9, 1187]), array([1284.18, 1284.29]), array([1302.65, 1302.8]), array([1316.06, 1316.27]), array([1693.34, 1693.48]), array([1725.28, 1725.6])])
        user_annotations[ctx + Ctx(decoder='long_RL')] = np.array([array([149.959, 150.254]), array([307.08, 307.194]), array([1332.28, 1332.39])])
        user_annotations[ctx + Ctx(decoder='short_LR')] = np.array([array([132.511, 132.791]), array([571.304, 571.385]), array([1284.18, 1284.29]), array([1302.65, 1302.8]), array([1316.06, 1316.27]), array([1699.23, 1699.36])])
        user_annotations[ctx + Ctx(decoder='short_RL')] = np.array([array([105.4, 105.563]), array([1302.65, 1302.8]), array([1332.28, 1332.39]), array([1450.89, 1451.02])])

    # inner_ctx = ctx.overwriting_context(epochs='ripple', decoder='short_RL')
    # inner_ctx = ctx + Ctx(epochs='ripple', decoder='short_RL')
    # print(f'inner_ctx: {inner_ctx}')
        
user_annotations

In [ ]:
# [4, 22, 24, 28, 30, 38, 42, 50, 55, 60, 67, 70, 76, 83, 85, 100, 103, 107, 108, 113, 118, 121, 122, 131, 140, 142, 149, 153, 170, 171]
# [2, 7, 11, 17, 20, 22, 30, 34, 38, 39, 41, 43, 47, 49, 55, 59, 60, 69, 70, 75, 77, 80, 83, 85, 86, 100, 107, 110, 113, 114, 115, 118, 120, 121, 122, 126, 130, 131, 138, 140, 142, 149, 157, 160, 168, 170]        
# [[380.739, 380.865], [550.845, 551.034], [600.244, 600.768], [1431.7, 1431.87], [2121.38, 2121.72]]
# [[1202.96, 1203.26], [1433.42, 1433.58], [1600.77, 1601.16], [1679.18, 1679.68]]
# [[551.872, 552.328], [565.161, 565.417], [616.348, 616.665], [919.581, 919.692], [1149.57, 1149.8], [1167.82, 1168.17], [1384.71, 1385.01], [1424.02, 1424.22], [1446.52, 1446.65], [1538.1, 1538.48], [1690.72, 1690.82], [1820.96, 1821.29], [1979.72, 1979.86], [1995.48, 1995.95], [2121.38, 2121.72], [2267.05, 2267.41]]
# [[373.508, 373.754], [391.895, 392.163], [600.244, 600.768], [1015.26, 1015.5], [1079.9, 1080.08], [1310.59, 1310.92], [1433.42, 1433.58], [1494.95, 1495.4], [1558.22, 1558.42], [1616.92, 1617.09], [1774.48, 1774.61], [1956.96, 1957.2], [2011.36, 2011.54], [2059.35, 2059.56], [2074.35, 2074.62], [2156.53, 2156.79], [2233.53, 2233.95], [2260.49, 2260.61], [2521.1, 2521.31]]

user_annotations = {}
# original_user_annotations
# with IdentifyingContext(display_fn_name='DecodedEpochSlices', user_annotation='selections') as annotations_kind_ctx:
with IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') as session_ctx:
    with (session_ctx + IdentifyingContext(display_fn_name='DecodedEpochSlices', user_annotation='selections')) as ctx:
        with (ctx + IdentifyingContext(epochs='replays')) as ctx:
            user_annotations[ctx + Ctx(decoder='long_results_obj')] = [4, 22, 24, 28, 30, 38, 42, 50, 55, 60, 67, 70, 76, 83, 85, 100, 103, 107, 108, 113, 118, 121, 122, 131, 140, 142, 149, 153, 170, 171]
            user_annotations[ctx + Ctx(decoder='short_results_obj')] = [2, 7, 11, 17, 20, 22, 30, 34, 38, 39, 41, 43, 47, 49, 55, 59, 60, 69, 70, 75, 77, 80, 83, 85, 86, 100, 107, 110, 113, 114, 115, 118, 120, 121, 122, 126, 130, 131, 138, 140, 142, 149, 157, 160, 168, 170]

        with (ctx + IdentifyingContext(epochs='ripple')) as ctx:
            user_annotations[ctx + Ctx(decoder='long_LR')] = [[380.739, 380.865], [550.845, 551.034], [600.244, 600.768], [1431.7, 1431.87], [2121.38, 2121.72]]
            user_annotations[ctx + Ctx(decoder='long_RL')] = [[1202.96, 1203.26], [1433.42, 1433.58], [1600.77, 1601.16], [1679.18, 1679.68]]
            user_annotations[ctx + Ctx(decoder='short_LR')] = [[551.872, 552.328], [565.161, 565.417], [616.348, 616.665], [919.581, 919.692], [1149.57, 1149.8], [1167.82, 1168.17], [1384.71, 1385.01], [1424.02, 1424.22], [1446.52, 1446.65], [1538.1, 1538.48], [1690.72, 1690.82], [1820.96, 1821.29], [1979.72, 1979.86], [1995.48, 1995.95], [2121.38, 2121.72], [2267.05, 2267.41]]
            user_annotations[ctx + Ctx(decoder='short_RL')] = [[373.508, 373.754], [391.895, 392.163], [600.244, 600.768], [1015.26, 1015.5], [1079.9, 1080.08], [1310.59, 1310.92], [1433.42, 1433.58], [1494.95, 1495.4], [1558.22, 1558.42], [1616.92, 1617.09], [1774.48, 1774.61], [1956.96, 1957.2], [2011.36, 2011.54], [2059.35, 2059.56], [2074.35, 2074.62], [2156.53, 2156.79], [2233.53, 2233.95], [2260.49, 2260.61], [2521.1, 2521.31]]

# Process raw annotations with the helper function
for context, sequences in user_annotations.items():
    user_annotations[context] = numpyify_array(sequences)
    
user_annotations

In [ ]:

for k,v in user_annotations.items():
    assert k in original_user_annotations
    assert (original_user_annotations[k] == v).all()
    # assert np.array_equal(original_user_annotations[k] == numpyify_array(v))


# user_annotations